In [ ]:
#Code answer for prompt n1


from bs4 import BeautifulSoup

# Load the HTML content of each file
file_paths = [
    "/mnt/data/Amazon.es_ Auriculares Inalambricos - Marca_ 10 Seleccionados_ Electrónica_1.html",
    "/mnt/data/Amazon.es_ Auriculares Inalambricos - Marca_ 10 Seleccionados_ Electrónica_2.html",
    "/mnt/data/Amazon.es_ Auriculares Inalambricos - Marca_ 10 Seleccionados_ Electrónica_3.html",
    "/mnt/data/Amazon.es_ Auriculares Inalambricos - Marca_ 10 Seleccionados_ Electrónica_4.html",
    "/mnt/data/Amazon.es_ Auriculares Inalambricos - Marca_ 10 Seleccionados_ Electrónica_5.html",
    "/mnt/data/Amazon.es_ Auriculares Inalambricos - Marca_ 10 Seleccionados_ Electrónica_6.html"
]

soup_objects = []

for path in file_paths:
    with open(path, 'r', encoding='utf-8') as file:
        content = file.read()
        soup = BeautifulSoup(content, 'html.parser')
        soup_objects.append(soup)

# Create a new BeautifulSoup object for the merged HTML
merged_soup = BeautifulSoup('<!DOCTYPE html><html lang="es"></html>', 'html.parser')
merged_html = merged_soup.html

# Merge the contents of all soup objects
for soup in soup_objects:
    body_content = soup.find('body')
    if body_content:
        merged_html.append(body_content)

# Output the merged HTML content
merged_html_content = merged_soup.prettify()

# Save the merged content to a new file
merged_file_path = "/mnt/data/Amazon.es_Auriculares_Inalambricos_Marca_10_Seleccionados_Electronica_Merged.html"
with open(merged_file_path, 'w', encoding='utf-8') as merged_file:
    merged_file.write(merged_html_content)

merged_file_path


In [ ]:
#Code answer for prompt n2

import pandas as pd
from bs4 import BeautifulSoup

# Load the merged HTML file
merged_file_path = "/mnt/data/Amazon.es_Auriculares_Inalambricos_Marca_10_Seleccionados_Electronica_Merged.html"
with open(merged_file_path, 'r', encoding='utf-8') as file:
    merged_content = file.read()
    soup = BeautifulSoup(merged_content, 'html.parser')

# Initialize lists to store scraped data
titles = []
prices = []
retail_prices = []
star_ratings = []
num_ratings = []
sold_last_month = []

# Extract the required information
for item in soup.find_all('div', class_='s-main-slot s-result-list s-search-results sg-row'):
    title = item.find('span', class_='a-size-base-plus a-color-base a-text-normal')
    price = item.find('span', class_='a-price')
    retail_price = item.find('span', class_='a-price a-text-price')
    star_rating = item.find('span', class_='a-icon-alt')
    num_rating = item.find('span', class_='a-size-base s-underline-text')
    sold_month = item.find('span', class_='a-size-base a-color-secondary')

    titles.append(title.text if title else None)
    prices.append(price.find('span', class_='a-offscreen').text if price else None)
    retail_prices.append(retail_price.find('span', class_='a-offscreen').text if retail_price else None)
    star_ratings.append(star_rating.text if star_rating else None)
    num_ratings.append(num_rating.text if num_rating else None)
    sold_last_month.append(sold_month.text if sold_month else None)

# Create a DataFrame
data = {
    'Title': titles,
    'Price': prices,
    'Recommended Retail Price': retail_prices,
    'Star Rating': star_ratings,
    'Number of Ratings': num_ratings,
    'Sold Last Month': sold_last_month
}
df = pd.DataFrame(data)

# Save the DataFrame to an Excel file
output_file_path = "/mnt/data/Amazon_Items_Scraped.xlsx"
df.to_excel(output_file_path, index=False)

import ace_tools as tools; tools.display_dataframe_to_user(name="Amazon Items Data", dataframe=df)

output_file_path


In [ ]:
#Code answer for prompt n3

# Renaming columns
data.rename(columns={
    'Title': 'product_description',
    'Price': 'price',
    'Recommended Retail Price': 'recommended_retail_price',
    'Star Rating': 'star_rating',
    'Number of Ratings': 'number_of_ratings',
    'Sold Last Month': 'units_sold_last_moth'
}, inplace=True)

# Cleaning 'price' column
data['price'] = data['price'].str.replace('€', '').str.replace(',', '.').astype(float)

# Cleaning 'recommended_retail_price' column
data['recommended_retail_price'] = data['recommended_retail_price'].str.replace('€', '').str.replace(',', '.').astype(float)

# Cleaning 'star_rating' column
data['star_rating'] = data['star_rating'].str.replace(' de 5 estrellas', '').str.replace(',', '.').astype(float)

# Cleaning 'units_sold_last_moth' column
data['units_sold_last_moth'] = data['units_sold_last_moth'].str.replace(' comprados el mes pasado', '')

# Display the first few rows of the cleaned data
import ace_tools as tools; tools.display_dataframe_to_user(name="Cleaned Amazon Wireless Earbuds Data", dataframe=data.head())

data.head()

In [ ]:
#Code answer for prompt n4

# Convert column to string before applying the function
data['units_sold_last_moth'] = data['units_sold_last_moth'].astype(str)

# Function to clean 'units_sold_last_moth' column
def clean_units_sold(value):
    value = value.replace('\xa0', ' ')
    if ' mil+' in value:
        return int(value.replace(' mil+', '')) * 1000
    elif '+' in value:
        return int(value.replace('+', ''))
    else:
        return ''
    
# Apply the cleaning function again
data['units_sold_last_moth'] = data['units_sold_last_moth'].apply(clean_units_sold)

# Display the cleaned dataframe to the user
tools.display_dataframe_to_user(name="Updated Amazon Wireless Earbuds Data with Cleaned Units Sold", dataframe=data.head())

# Save the updated dataframe to an Excel file
updated_file_path = '/mnt/data/Updated_Amazon_Wireless_Earbuds_Data.xlsx'
data.to_excel(updated_file_path, index=False)

updated_file_path   


In [ ]:
# Code answer for prompt n5

# List of brands
brands_list = ['Xiaomi', 'Sony', 'JBL', 'HUAWEI', 'Philips', 'JVC', 'Apple', 'Beats by Dr. Dre', 'Energy Sistem', 'LG']

# Function to find the brand in the product description
def find_brand(description):
    for brand in brands_list:
        if brand in description:
            return brand
    return 'Other'

# Apply the function to create a new column 'brands'
data['brands'] = data['product_description'].apply(find_brand)

# Display the updated dataframe to the user
tools.display_dataframe_to_user(name="Amazon Wireless Earbuds Data with Brands", dataframe=data.head())

# Save the updated dataframe to an Excel file
final_file_path = '/mnt/data/Amazon_Wireless_Earbuds_Data_with_Brands.xlsx'
data.to_excel(final_file_path, index=False)

final_file_path


In [ ]:
# Code answer for prompt n6

# Define the keywords for each category
device_life_keywords = ['battery', 'autonomy', 'bateria', 'autonomia', 'horas', 'hours']
noise_cancel_keywords = ['cancelacion', 'ruido', 'noise', 'cancellation', 'supresion', 'supression']
fast_charging_keywords = ['carga rapida', 'fast charging']
compatibility_keywords = ['compatibles', 'compatible']

# Function to check for keywords in product_description and return 'yes' or 'no'
def check_keywords(description, keywords):
    return 'yes' if any(keyword in description.lower() for keyword in keywords) else 'no'

# Apply the function to create the new columns
data['device_life'] = data['product_description'].apply(lambda x: check_keywords(x, device_life_keywords))
data['noise_cancel'] = data['product_description'].apply(lambda x: check_keywords(x, noise_cancel_keywords))
data['fast_charging'] = data['product_description'].apply(lambda x: check_keywords(x, fast_charging_keywords))
data['compatibility'] = data['product_description'].apply(lambda x: check_keywords(x, compatibility_keywords))

# Create the 'punctuation' column by summing the 'yes' values in the four new columns
data['punctuation'] = data[['device_life', 'noise_cancel', 'fast_charging', 'compatibility']].apply(lambda x: x.str.count('yes')).sum(axis=1)

import ace_tools as tools; tools.display_dataframe_to_user(name="Amazon Wireless Earbuds Data with Classification", dataframe=data)

# Display the updated dataframe
data.head()


In [ ]:
# Code answer for prompt n7

# First, remove exact duplicates based on product_description
total_rows_before = len(data)
data.drop_duplicates(subset='product_description', inplace=True)
total_rows_after = len(data)
number_of_duplicates = total_rows_before - total_rows_after

# Create a new column 'product_id'
data['product_id'] = data.groupby('brands').cumcount() + 1
data['product_id'] = data['brands'] + '_' + data['product_id'].astype(str)

import ace_tools as tools; tools.display_dataframe_to_user(name="Amazon Wireless Earbuds Data with Product ID", dataframe=data)

# Display the number of duplicates and the total number of rows before and after the scan
total_rows_before, total_rows_after, number_of_duplicates


In [ ]:
# Code answer for prompt n8

# Updating the brand names based on product_id
data.loc[data['product_id'] == 'Other_1', 'brands'] = 'Xiaomi'
data.loc[data['product_id'] == 'Other_2', 'brands'] = 'Philips'
data.loc[data['product_id'] == 'Other_3', 'brands'] = 'Philips'
data.loc[data['product_id'] == 'Other_4', 'brands'] = 'Beats by Dr. Dre'
data.loc[data['product_id'] == 'Other_5', 'brands'] = 'Philips'
data.loc[data['product_id'] == 'Other_6', 'brands'] = 'Philips'
data.loc[data['product_id'] == 'Other_7', 'brands'] = 'Sony'

# Updating brand names for product descriptions beginning with 'Beats'
data.loc[data['product_description'].str.startswith('Beats'), 'brands'] = 'Beats by Dr. Dre'

# Updating the product_id for changed brand entries
data['product_id'] = data.groupby('brands').cumcount() + 1
data['product_id'] = data['brands'] + '_' + data['product_id'].astype(str)

import ace_tools as tools; tools.display_dataframe_to_user(name="Updated Brand Data", dataframe=data)

# Display the updated data
data[data['brands'].isin(['Xiaomi', 'Philips', 'Beats by Dr. Dre', 'Sony'])]


In [ ]:
# Code answer for prompt n9

# Counting the number of items per brand and sorting them from top to bottom
items_per_brand_sorted = data['brands'].value_counts().reset_index()
items_per_brand_sorted.columns = ['brands', 'item_count']

# Displaying the results
import ace_tools as tools; tools.display_dataframe_to_user(name="Items Per Brand Sorted", dataframe=items_per_brand_sorted)

items_per_brand_sorted


In [ ]:
# Code answer for prompt n10

# Filtering the dataframe to include only brands with more than 10 items
brands_to_analyze = items_per_brand_sorted[items_per_brand_sorted['item_count'] > 10]['brands']

filtered_data = data[data['brands'].isin(brands_to_analyze)]

import ace_tools as tools; tools.display_dataframe_to_user(name="Filtered Data for Brands with More Than 10 Items", dataframe=filtered_data)

filtered_data.head()


In [ ]:
# Code answer for prompt n11

# Calculate the mean and median values for each column and brand
means = filtered_data.groupby('brands').mean()
medians = filtered_data.groupby('brands').median()

# Dictionary to store the mean and median values used for each iteration
imputation_values = {
    'price': {},
    'recommended_retail_price': {},
    'units_sold_last_moth': {}
}

# Function to replace missing values based on mean or median
def replace_missing_values(df, column, brand, method):
    if method == 'mean':
        value = means.loc[brand, column]
    elif method == 'median':
        value = medians.loc[brand, column]
    else:
        raise ValueError("Method must be either 'mean' or 'median'")
    
    df.loc[df['brands'] == brand, column] = df.loc[df['brands'] == brand, column].fillna(value)
    return value

# Perform the replacements as specified
brands = ['Beats by Dr. Dre', 'Sony', 'JBL', 'Xiaomi']

# Price replacements
imputation_values['price']['Beats by Dr. Dre'] = replace_missing_values(filtered_data, 'price', 'Beats by Dr. Dre', 'mean')
imputation_values['price']['Sony'] = replace_missing_values(filtered_data, 'price', 'Sony', 'mean')
imputation_values['price']['JBL'] = replace_missing_values(filtered_data, 'price', 'JBL', 'median')
imputation_values['price']['Xiaomi'] = replace_missing_values(filtered_data, 'price', 'Xiaomi', 'median')

# Recommended Retail Price replacements
imputation_values['recommended_retail_price']['Beats by Dr. Dre'] = replace_missing_values(filtered_data, 'recommended_retail_price', 'Beats by Dr. Dre', 'mean')
imputation_values['recommended_retail_price']['Sony'] = replace_missing_values(filtered_data, 'recommended_retail_price', 'Sony', 'mean')
imputation_values['recommended_retail_price']['Xiaomi'] = replace_missing_values(filtered_data, 'recommended_retail_price', 'Xiaomi', 'mean')
imputation_values['recommended_retail_price']['JBL'] = replace_missing_values(filtered_data, 'recommended_retail_price', 'JBL', 'median')

# Units Sold Last Month replacements
imputation_values['units_sold_last_moth']['Beats by Dr. Dre'] = replace_missing_values(filtered_data, 'units_sold_last_moth', 'Beats by Dr. Dre', 'mean')
imputation_values['units_sold_last_moth']['Sony'] = replace_missing_values(filtered_data, 'units_sold_last_moth', 'Sony', 'median')
imputation_values['units_sold_last_moth']['Xiaomi'] = replace_missing_values(filtered_data, 'units_sold_last_moth', 'Xiaomi', 'median')
imputation_values['units_sold_last_moth']['JBL'] = replace_missing_values(filtered_data, 'units_sold_last_moth', 'JBL', 'median')

import ace_tools as tools; tools.display_dataframe_to_user(name="Imputation Values Used", dataframe=pd.DataFrame(imputation_values))

# Display the imputation values used
imputation_values


In [ ]:
# Code answer for prompt n12

# Calculate the mean of star_rating and number_of_ratings by brand
brand_star_rating_mean = data.groupby('brands')['star_rating'].mean().reset_index()
brand_number_of_ratings_mean = data.groupby('brands')['number_of_ratings'].mean().reset_index()

# Create a figure with two subplots
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 6))

# Bar graph for mean star_rating by brand
axes[0].barh(brand_star_rating_mean['brands'], brand_star_rating_mean['star_rating'], color='skyblue')
axes[0].set_title('Mean Star Rating by Brand')
axes[0].set_xlabel('Mean Star Rating')
axes[0].set_ylabel('Brands')
axes[0].invert_yaxis()  # Invert y axis to match the order in the second graph

# Bar graph for mean number of ratings by brand
axes[1].barh(brand_number_of_ratings_mean['brands'], brand_number_of_ratings_mean['number_of_ratings'], color='salmon')
axes[1].set_title('Mean Number of Ratings by Brand')
axes[1].set_xlabel('Mean Number of Ratings')
axes[1].set_ylabel('Brands')
axes[1].invert_yaxis()  # Invert y axis to match the order in the first graph

plt.tight_layout()
plt.show()


In [ ]:
# Code answer to promtp n13:

# Calculate the total units sold last month by brand and sort from highest to lowest
brand_units_sold = data.groupby('brands')['units_sold_last_moth'].sum().sort_values(ascending=False).reset_index()

# Get the top 10 products sold last month and sort by units sold
top_products_sold = data[['product_id', 'units_sold_last_moth']].sort_values(by='units_sold_last_moth', ascending=False).head(10)

# Create a figure with two subplots
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(18, 8))

# Bar graph of units_sold_last_month by brand
axes[0].barh(brand_units_sold['brands'], brand_units_sold['units_sold_last_moth'], color='lightgreen')
axes[0].set_title('Units Sold Last Month by Brand')
axes[0].set_xlabel('Units Sold Last Month')
axes[0].set_ylabel('Brands')
axes[0].invert_yaxis()  # Invert y axis to show top brands at the top

# Bar graph of top 10 products sold last month
axes[1].barh(top_products_sold['product_id'], top_products_sold['units_sold_last_moth'], color='lightcoral')
axes[1].set_title('Top 10 Products Sold Last Month')
axes[1].set_xlabel('Units Sold Last Month')
axes[1].set_ylabel('Product ID')
axes[1].invert_yaxis()  # Invert y axis to show top products at the top

plt.tight_layout()
plt.show()


In [ ]:
# Code answer for prompt n14

import numpy as np

# Calculate mean units sold last month by brand
mean_units_by_brand = data.groupby('brands')['units_sold_last_moth'].mean()
# Calculate mean star rating by brand
mean_star_by_brand = data.groupby('brands')['star_rating'].mean()

# Set distinct and clear colors for each brand
brand_colors = {'JBL': 'darkblue', 'Beats by Dr. Dre': 'darkred', 'Xiaomi': 'darkgreen', 'Sony': 'darkviolet'}

# Scatter plot for correlation between star_rating and units_sold_last_month
plt.figure(figsize=(12, 8))
for brand, group_data in data.groupby('brands'):
    plt.scatter(group_data['star_rating'], group_data['units_sold_last_moth'], 
                color=brand_colors[brand], label=brand, alpha=0.6, edgecolors='black')

# Add dashed lines for mean values by brand
for brand in mean_units_by_brand.index:
    plt.axhline(y=mean_units_by_brand[brand], color=brand_colors[brand], linestyle='--', linewidth=1)
    plt.axvline(x=mean_star_by_brand[brand], color=brand_colors[brand], linestyle='--', linewidth=1)

plt.title('Correlation between Star Rating and Units Sold Last Month')
plt.xlabel('Star Rating')
plt.ylabel('Units Sold Last Month')
plt.legend(title='Brand', loc='center left', bbox_to_anchor=(1, 0.5))
plt.grid(True)
plt.show()


In [ ]:
# Code answer to prompt n15

# Calculate mean number of ratings by brand
mean_ratings_by_brand = data.groupby('brands')['number_of_ratings'].mean()

# Scatter plot for correlation between star_rating and number_of_ratings
plt.figure(figsize=(12, 8))
for brand, group_data in data.groupby('brands'):
    plt.scatter(group_data['star_rating'], group_data['number_of_ratings'], 
                color=brand_colors[brand], label=brand, alpha=0.6, edgecolors='black')

# Add dashed lines for mean values by brand
for brand in mean_ratings_by_brand.index:
    plt.axhline(y=mean_ratings_by_brand[brand], color=brand_colors[brand], linestyle='--', linewidth=1)
    plt.axvline(x=mean_star_by_brand[brand], color=brand_colors[brand], linestyle='--', linewidth=1)

plt.title('Correlation between Star Rating and Number of Ratings')
plt.xlabel('Star Rating')
plt.ylabel('Number of Ratings')
plt.legend(title='Brand', loc='center left', bbox_to_anchor=(1, 0.5))
plt.grid(True)
plt.show()


In [ ]:
# Code answer to prompt n16:

# Create a scatter plot for the correlation between price and units_sold_last_month with reversed color scale
plt.figure(figsize=(12, 8))
sc = plt.scatter(data['price'], data['units_sold_last_moth'], 
                 c=data['star_rating'], cmap='viridis_r', alpha=0.6, edgecolors='black')

# Add color bar to represent the star rating scale
cbar = plt.colorbar(sc)
cbar.set_label('Star Rating')

plt.title('Correlation between Price and Units Sold Last Month with Star Rating as Color Scale')
plt.xlabel('Price')
plt.ylabel('Units Sold Last Month')
plt.grid(True)
plt.show()


In [ ]:
# Code answer for prompt n17

# Create a figure with subplots for each brand with a grey background
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(16, 12))

# Flatten the axes array for easier iteration
axes = axes.flatten()

# Iterate over each brand and create a scatter plot for each
for idx, (brand, group_data) in enumerate(data.groupby('brands')):
    ax = axes[idx]
    ax.set_facecolor('lightgrey')
    sc = ax.scatter(group_data['price'], group_data['units_sold_last_moth'], 
                    c=group_data['star_rating'], cmap=brand_color_maps[brand], alpha=0.6, edgecolors='black')
    ax.set_title(f'Price vs. Units Sold for {brand}')
    ax.set_xlabel('Price')
    ax.set_ylabel('Units Sold Last Month')
    cbar = plt.colorbar(sc, ax=ax)
    cbar.set_label('Star Rating')
    ax.grid(True)

plt.tight_layout()
plt.show()


In [6]:
# Code to be able to see screenshots and download the file to pdf (not from ChatGPT)

from IPython.display import display
from PIL import Image
